Copyright **`(c)`** 2023 Arturo Adelfio (s316716)

`not completed due to time constraints`

# LAB10

Use reinforcement learning to devise a tic-tac-toe player.

### Deadlines:

* Submission: [Dies Natalis Solis Invicti](https://en.wikipedia.org/wiki/Sol_Invictus)
* Reviews: [Befana](https://en.wikipedia.org/wiki/Befana)

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [187]:
from itertools import combinations
from collections import namedtuple, defaultdict
from random import choice
from copy import deepcopy

from tqdm.auto import tqdm
import numpy as np

In [188]:
State = namedtuple('State', ['x', 'o'])

In [189]:
MAGIC = [2, 7, 6, 9, 5, 1, 4, 3, 8]

In [190]:
def print_board(pos):
    """Nicely prints the board"""
    for r in range(3):
        for c in range(3):
            i = r * 3 + c
            if MAGIC[i] in pos.x:
                print('X', end='')
            elif MAGIC[i] in pos.o:
                print('O', end='')
            else:
                print('.', end='')
        print()
    print()

In [191]:
def win(elements):
    """Checks is elements is winning"""
    return any(sum(c) == 15 for c in combinations(elements, 3))

def state_value(pos: State):
    """Evaluate state: +1 first player wins"""
    if win(pos.x):
        return 1 #first player win
    elif win(pos.o):
        return -1 #second player win
    else:
        return 0 #drawn

In [192]:
class RandomPlayer:
    def __init__(self) -> None:
        pass
        
    def move(self,possible_moves, state=None):
        return choice(list(possible_moves))

In [193]:
import random

class QAgent:
    def __init__(self, learning_rate=0.1, discount_factor=0.9, exploration_prob=1.0, exploration_decay=0.995):
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_prob = exploration_prob
        self.exploration_decay = exploration_decay
        # Initialize Q-table 
        self.q_table = {}
        self.training = True

    def move(self, possible_moves, state):
        # Exploration-exploitation trade-off
        #print(self.q_table)
        if np.random.rand() < self.exploration_prob: #Explore
            return choice(list(possible_moves))
        else: #Exploit
            #Choosing from the QTable
            action = np.argmax(self.q_table[state, :]) 
        
        
            
        return action
        

    def update_q_table(self, state, action, reward, next_state):
        # Convert next_state to frozenset if it's a set
        print("Next state", next_state)
        next_state = (frozenset(next_state.x), frozenset(next_state.o))
        state = (frozenset(state.x), frozenset(state.o))

        # Q-value update using the Bellman equation
        print(self.q_table)
        if next_state not in self.q_table:
            self.q_table[next_state] = np.zeros((10,))
        if state not in self.q_table:
            self.q_table[state] = np.zeros((10,))
        
        current_q_value = self.q_table[state][action]
        max_future_q_value = np.max(self.q_table[next_state])
        new_q_value = (1 - self.learning_rate) * current_q_value + \
                    self.learning_rate * (reward + self.discount_factor * max_future_q_value)
        self.q_table[state][action] = new_q_value

        

In [194]:
def game(p1,p2, train = True):
    trajectory = list()
    state = State(set(), set())
    available = set(range(1, 9+1))
    
    
    game_hist = list()
    players = []
    moves=list()
    players.append(p1)
    players.append(p2)
    index = 0
    i = 0
    current_player = players[index]
    agent = None
    
    while True:
        #print(current_player, index)
        
        if index == 0:
            move = current_player.move(available, state)
           
            #print(available)
            #print(move)
            state.x.add(move)
            
            moves.append(move)
            trajectory.append(deepcopy(state))
            available.remove(move)
            
            if win(state.x) or not available:
                break
        else:
            move = current_player.move(available, state)
            state.o.add(move)
            moves.append(move)
            trajectory.append(deepcopy(state))
            available.remove(move)
            #print(available)
            #print(move)
            if win(state.o) or not available:
                break
        
        
        isAgent = isinstance(current_player, QAgent)
        
        if isAgent and agent == None:
            agent = current_player
            previous_action = move
        
        if not isAgent  and i > 2: #QAgent opponent is playing, but it's not the first move, so QAgent already played
            #print(trajectory)
            previous_state = trajectory[-2]
            #print("previous state",previous_state)
            #print("reward", reward)
            #print("last_move", moves[-2])
            reward = moves[-2] // 3
                        
            agent.update_q_table(previous_state, previous_action, reward, state)
        
        i = i+1
            
        
            
        index = 1 - index
        current_player = players[index]
        result = state_value(state)
        #print("result", result) #1 first winning, -1 second winning, 0 drawn
        
    return trajectory, moves, result, available

In [195]:
#game()

In [196]:

def train():
    player1 = RandomPlayer()
    player0 = QAgent()
    qAgent_wins = 0
    qAgent_lost = 0
    games_number = 100
    
    for _ in tqdm(range(games_number)):
        trajectory, moves, reward, possible_moves = game(player0, player1, train=True)
        
        if reward == 1: #first player (QAgent wins)
            qAgent_wins += 1
            reward = reward + 10
            state = trajectory[-1]
            move = moves[-1]
            #new_state = deepcopy(state)
            #new_state.x.add(choice(possible_moves))
        elif reward == -1:
            reward == reward -20
            qAgent_lost +=1
            state = trajectory[-2]
            move = moves[-2]
            new_state = trajectory[-1]
        else:
            state = trajectory[-1]
            move = moves[-1]
            #new_state = deepcopy(state)
            #new_state.x.add(choice(possible_moves))
        
        
        
        
        new_state = trajectory[-1]
        
            
        player0.update_q_table(state, move, reward, new_state)
        
    
     
    print("Winning/Draw", (games_number - qAgent_lost)/games_number)
    print("Lost games", qAgent_lost/games_number)   

        
            
            
    

In [ ]:
train()